# Script to detect faces using dlib.

### Importing libraries

In [11]:
import gc
import os
import sys
import math
import glob
import tqdm
import random
import numpy as np
from tqdm import tqdm
from time import sleep

In [12]:
import pandas as pd
import xml.etree.cElementTree as ET

In [13]:
import cv2
import dlib
from imutils import face_utils
from skimage.feature import hog
from skimage import data,exposure

### Defining paths

In [14]:
path_folder_all_frames = '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_all_in_one'

In [15]:
path_folder = '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise'

## Write an xml file, given the folder of images and path for detector(optional)

### Generating an xml file

#### From folder structure 
    folder_data
        folder_video_0
            video_0_frame_1.jpg
            video_0_frame_2.jpg
            video_0_frame_3.jpg
        folder_video_1
            video_0_frame_1.jpg
            video_0_frame_2.jpg
            video_0_frame_3.jpg
        .
        .
        .

In [16]:
def writeXMLForDetectedFaces(path_folder_of_folders_of_images, detector,predictor, output_file_name = 'output',isCNN=False):
    """
    writes XML after detecting faces given source folder of video folders containing image frames, a landmark predictor and a face detector
    """
    
    #Writing an XML
    root = ET.Element("dataset")
    name = ET.SubElement(root, "name").text = "Labelled faces"
    comment = ET.SubElement(root, "comment").text = "These are labelled images from Bagamoyo"
    images = ET.SubElement(root, "images")
    
    list_folder_images = glob.glob(path_folder_of_folders_of_images + '/*')
    print("Number of folders found in the folder are: ", len(list_folder_images))
    for vid_no,vid_folder in enumerate(list_folder_images):
        list_path_frames = glob.glob(vid_folder+'/*')
        print("Number of images found in the folder number {}, {} are: ".format(vid_no,vid_folder), len(list_path_frames))
        try:
            for path_frame in list_path_frames:
                #add image to images in XML
                image_node = ET.SubElement(images, "image", file=path_frame.split('/')[-1])
#                 print(path_frame)
#             try:
                image=cv2.imread(path_frame)
                gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
                    
                if(isCNN):
                    dets = detector(gray,1)
                    rects = dlib.rectangles()
                    rects.extend([d.rect for d in dets])
                else:
                    rects = detector(gray,1)
                for i,rect in enumerate(rects):                    
                    shape = predictor(gray, rect)
                    shape = face_utils.shape_to_np(shape)
                    (x, y, w, h) = face_utils.rect_to_bb(rect)
                    #add box to image in XML
#                     print(x,y,w,h)
                    box = ET.SubElement(image_node, "box", height=str(h), left=str(x), top=str(y), width=str(w))
                #write XML
            tree = ET.ElementTree(root)
            dest_name = output_file_name+str(vid_no)+'.xml'
            tree.write(dest_name)
            print("xml written to {}".format(dest_name))            
        except KeyboardInterrupt:
            break
        except:
            continue

In [5]:
def saveXMLForHOGDetector(path_folder_of_folders_of_images,output_file_name='output'):
    args={"shape_predictor":"shape_predictor_68_face_landmarks.dat"}
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(args["shape_predictor"])
    writeXMLForDetectedFaces(path_folder_of_folders_of_images=path_folder_of_folders_of_images, detector=detector,predictor=predictor,output_file_name = output_file_name)

In [17]:
def saveXMLForCustomHOGDetector(path_detector, path_folder_of_folders_of_images,output_file_name='output'):
    args={"shape_predictor":"shape_predictor_68_face_landmarks.dat"}
    detector = dlib.simple_object_detector(path_detector)
    predictor = dlib.shape_predictor(args["shape_predictor"])
    writeXMLForDetectedFaces(path_folder_of_folders_of_images=path_folder_of_folders_of_images, detector=detector,predictor=predictor,output_file_name = output_file_name)

In [47]:
def saveXMLForCNNDetector(path_folder_of_folders_of_images,output_file_name='output'):
    args={"shape_predictor":"shape_predictor_68_face_landmarks.dat"}
    detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')
    predictor = dlib.shape_predictor(args["shape_predictor"])
    writeXMLForDetectedFaces(path_folder_of_folders_of_images=path_folder_of_folders_of_images, detector=detector,predictor=predictor,output_file_name = output_file_name,isCNN=True)

In [18]:
if __name__ == "__main__":
    saveXMLForCustomHOGDetector('analysis/detector1.svm',path_folder,'custom_hog_output_1')

('Number of folders found in the folder are: ', 524)
('Number of images found in the folder number 0, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_00-male-surprise_20180516_091529_219393302 (5-29-2018 10-29-50 AM) are: ', 104)
xml written to custom_hog_output_10.xml
('Number of images found in the folder number 1, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_00-male-surprise_20180516_091541_1754978908 (5-27-2018 5-58-50 AM) are: ', 107)
xml written to custom_hog_output_11.xml
('Number of images found in the folder number 2, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_00-male-surprise_20180516_091551_1272027070 (5-29-2018 10-32-30 AM) are: ', 134)
xml written to custom_hog_output_12.xml
('Number of images found in the folder number 3, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_00-male-surprise_20180516_091601_315735604 (5-27-2018 5-40-04 AM) are:

xml written to custom_hog_output_132.xml
('Number of images found in the folder number 33, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_03-female-delight_20180516_103833_24953007 (5-29-2018 10-26-34 AM) are: ', 150)
xml written to custom_hog_output_133.xml
('Number of images found in the folder number 34, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_03-female-delight_20180516_104052_1280076545 (5-29-2018 10-05-58 AM) are: ', 300)
xml written to custom_hog_output_134.xml
('Number of images found in the folder number 35, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_03-female-delight_20180516_104413_411561700 (5-27-2018 6-00-56 AM) are: ', 127)
xml written to custom_hog_output_135.xml
('Number of images found in the folder number 36, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_03-female-delight_20180516_104906_1866004401 (5-27-2018 5-33-22 AM) are: '

xml written to custom_hog_output_165.xml
('Number of images found in the folder number 66, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_03-female-delight_20180517_050657_352334150 (5-27-2018 5-40-50 AM) are: ', 130)
xml written to custom_hog_output_166.xml
('Number of images found in the folder number 67, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_03-female-delight_20180517_051119_1871581760 (5-27-2018 6-02-16 AM) are: ', 257)
xml written to custom_hog_output_167.xml
('Number of images found in the folder number 68, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_03-female-delight_20180517_111312_437693398 (5-27-2018 5-29-36 AM) are: ', 131)
xml written to custom_hog_output_168.xml
('Number of images found in the folder number 69, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_03-female-delight_20180517_111514_642995551 (5-29-2018 10-31-40 AM) are: ',

xml written to custom_hog_output_198.xml
('Number of images found in the folder number 99, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_06-female-confusion_20180517_050404_2079835790 (5-27-2018 5-45-26 AM) are: ', 138)
xml written to custom_hog_output_199.xml
('Number of images found in the folder number 100, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_06-female-confusion_20180517_050855_852365877 (5-24-2018 8-12-44 PM) are: ', 150)
xml written to custom_hog_output_1100.xml
('Number of images found in the folder number 101, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_06-female-confusion_20180517_051057_1603931698 (5-27-2018 5-35-04 AM) are: ', 135)
xml written to custom_hog_output_1101.xml
('Number of images found in the folder number 102, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_06-female-confusion_20180517_111249_1387834407 (5-27-2018 7-25-

xml written to custom_hog_output_1131.xml
('Number of images found in the folder number 132, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_10-male-confusion_20180517_045050_1194525893 (5-27-2018 5-33-56 AM) are: ', 147)
xml written to custom_hog_output_1132.xml
('Number of images found in the folder number 133, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_10-male-confusion_20180517_045323_956819529 (5-27-2018 5-31-36 AM) are: ', 102)
xml written to custom_hog_output_1133.xml
('Number of images found in the folder number 134, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_10-male-confusion_20180517_045606_1194917364 (5-27-2018 6-04-00 AM) are: ', 139)
xml written to custom_hog_output_1134.xml
('Number of images found in the folder number 135, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_10-male-confusion_20180517_045916_145201778 (5-29-2018 10-06-38 AM

...

xml written to custom_hog_output_1388.xml
('Number of images found in the folder number 389, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_male-delight_20180516_105347_114485686 (5-27-2018 5-31-06 AM) are: ', 106)
xml written to custom_hog_output_1389.xml
('Number of images found in the folder number 390, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_male-delight_20180516_105521_1689307720 (5-29-2018 10-25-56 AM) are: ', 110)
xml written to custom_hog_output_1390.xml
('Number of images found in the folder number 391, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_male-delight2_20180516_091358_421238878 (5-27-2018 6-00-42 AM) are: ', 110)
xml written to custom_hog_output_1391.xml
('Number of images found in the folder number 392, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_male-delight2_20180516_091801_70387599 (5-29-2018 10-28-34 AM) are: ', 132)
xml 

xml written to custom_hog_output_1421.xml
('Number of images found in the folder number 422, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_08-female-frustration_20180516_103848_514441617 (5-29-2018 10-28-08 AM) are: ', 239)
xml written to custom_hog_output_1422.xml
('Number of images found in the folder number 423, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_08-female-frustration_20180516_104118_440052272 (5-27-2018 5-38-38 AM) are: ', 272)
xml written to custom_hog_output_1423.xml
('Number of images found in the folder number 424, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_08-female-frustration_20180516_104427_354754090 (5-29-2018 10-31-06 AM) are: ', 159)
xml written to custom_hog_output_1424.xml
('Number of images found in the folder number 425, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_08-female-frustration_20180516_104924_1779673513 (5-27

xml written to custom_hog_output_1453.xml
('Number of images found in the folder number 454, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_1526475586855 confused_20180517_050049_1004309012 (5-24-2018 1-38-50 PM) are: ', 165)
xml written to custom_hog_output_1454.xml
('Number of images found in the folder number 455, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_1526475586855 confused_20180517_050323_273860978 (5-29-2018 10-06-52 AM) are: ', 147)
xml written to custom_hog_output_1455.xml
('Number of images found in the folder number 456, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_1526475586855 confused_20180517_050601_1440658405 (5-29-2018 10-25-26 AM) are: ', 85)
xml written to custom_hog_output_1456.xml
('Number of images found in the folder number 457, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_1526475586855 confused_20180517_050813_1836851824 

xml written to custom_hog_output_1485.xml
('Number of images found in the folder number 486, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_male-delight_20180516_110905_149443334 (5-27-2018 5-32-14 AM) are: ', 91)
xml written to custom_hog_output_1486.xml
('Number of images found in the folder number 487, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_male-delight_20180517_044309_618815639 (5-27-2018 5-38-08 AM) are: ', 151)
xml written to custom_hog_output_1487.xml
('Number of images found in the folder number 488, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_male-delight_20180517_044514_422746481 (5-29-2018 10-26-16 AM) are: ', 399)
xml written to custom_hog_output_1488.xml
('Number of images found in the folder number 489, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_male-delight_20180517_044754_536584609 (5-29-2018 10-28-16 AM) are: ', 113)
xml wri

xml written to custom_hog_output_1518.xml
('Number of images found in the folder number 519, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_male-delight2_20180516_105142_1199185530 (5-27-2018 5-42-48 AM) are: ', 217)
xml written to custom_hog_output_1519.xml
('Number of images found in the folder number 520, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_male-delight2_20180517_044738_2083257416 (5-27-2018 5-41-56 AM) are: ', 181)
xml written to custom_hog_output_1520.xml
('Number of images found in the folder number 521, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_male-delight_20180516_081759_199585785 (5-27-2018 5-45-20 AM) are: ', 105)
xml written to custom_hog_output_1521.xml
('Number of images found in the folder number 522, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_male-delight_20180516_105717_1111514052 (5-27-2018 6-04-12 AM) are: ', 245)
xml

In [49]:
saveXMLForCNNDetector(path_folder,'cnn_output')

RuntimeError: Unable to open mmod_human_face_detector.dat for reading.

In [12]:
cnn_face_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')

In [ ]:
writeXMLForDetectedFaces()

Writing XML for Histogram normalised

In [14]:
path_histogram_images = '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_histogram_normalised/'

In [15]:
file_names = [path_histogram_images+os.path.basename(f) for f in glob.glob(path_folder_all_frames+'/*.jpg')]

In [16]:
path_output_xml = "/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_histogram_normalised_output/"

In [17]:
args={"shape_predictor":"shape_predictor_68_face_landmarks.dat"}
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(args["shape_predictor"])

In [ ]:
list_path_frames = file_names
print("Number of images found are:{} ".format(len(list_path_frames)))
# Writing an XML
root = ET.Element("dataset")
name = ET.SubElement(root, "name").text = "Labelled faces"
comment = ET.SubElement(root, "comment").text = "These are labelled images from Bagamoyo"
images = ET.SubElement(root, "images")
for j,path_frame in enumerate(list_path_frames):
#     print(i,path_frame)
    if(j > 18233)
        try:
        #add image to images in XML
            image_node = ET.SubElement(images, "image", file=path_frame.split('/')[-1])
        #                 print(path_frame)
    #                 try:
    #         os.path.e
            gray=cv2.imread(path_frame)
        #         gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

        #         if(isCNN):
        #             dets = detector(gray,1)
        #             rects = dlib.rectangles()
        #             rects.extend([d.rect for d in dets])
        #         else:
    #         print(gray)
            rects = detector(gray,1)
    #         print(rects)
            for i,rect in enumerate(rects):                    
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                (x, y, w, h) = face_utils.rect_to_bb(rect)
                #add box to image in XML
    #             print(x,y,w,h)
                box = ET.SubElement(image_node, "box", height=str(h), left=str(x), top=str(y), width=str(w))
            #write XML
    #         print(j%5)
            if(j%1000==0):
                tree = ET.ElementTree(root)
                dest_name = path_output_xml+str(j)+'.xml'
                tree.write(dest_name)
                print("xml written to {}".format(dest_name))            
        except KeyboardInterrupt:
            break
        except:
            continue

Number of images found are:44492 
xml written to /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_histogram_normalised_output/19000.xml
xml written to /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_histogram_normalised_output/20000.xml
xml written to /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_histogram_normalised_output/21000.xml
xml written to /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_histogram_normalised_output/22000.xml
xml written to /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_histogram_normalised_output/23000.xml
xml written to /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_histogram_normalised_output/24000.xml
xml written to /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_histogram_normalised_output/25000.xml
xml written to /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_histogram_normalised_output/26000.xml
xml written to /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_histogram_normalised_output/2700

In [49]:
! ls /media/amogh/Stuff/CMU/datasets/bagamoyo_histogram_normalised_output/

ls: cannot access '/media/amogh/Stuff/CMU/datasets/bagamoyo_histogram_normalised_output/': No such file or directory
